# Telecom Data Formatting Notebook

This notebook demonstrates a general method to process telecom sector CSV files, combine rows based on a key column, aggregate data, reformat it, and write it to a styled Excel workbook.

The workflow:

1. Load the raw CSV file.
2. Group rows by a key column (`record_id` in our example).
3. Aggregate repeated columns and sum bandwidths by technology type.
4. Rename and reorder columns according to the final template.
5. Apply Excel formatting (header colors, bold font).
6. Save the result in a multi-sheet workbook for multiple companies.


In [13]:
# Import necessary libraries
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
import os


## Example: Raw Data


| licence_category* | area_name* | technology | record_id | bandwidth | ... |
|-------------------|------------|------------|-----------|-----------|-----|
| AWS               | Quebec     | LTE        | E5015     | 10        | ... |
| AWS               | Quebec     | LTE        | E5015     | 10        | ... |
| AWS               | Quebec     | LTE        | E5015     | 10        | ... |
| AWS               | Quebec     | LTE        | E0092     | 10        | ... |
| AWS               | Quebec     | LTE        | E0092     | 10        | ... |



In [14]:
# Example input file
input_csv = "Rogers sectors v3 Final.csv"

# Read everything from the CSV
df_raw = pd.read_csv(input_csv)

# Display first 5 rows
#df_raw.head()


## Data reformatting 
The following function takes a CSV file and outputs a formatted worksheet in an Excel file.  


- `record_id` represents the tower id and is the column used for grouping.
- `latitude`, `longitude`, `structure_height` are unique per tower and occupy a column in the output table
- Columns like `licence_category*`, `technology`, `bandwidth` may repeat per `record_id`, and they are aggregated into a single column in the output table



Some new columns are introduced in the output table. 
For example:

- Bandwidth is the sum of the "" column. 
- Adds derived columns like `Type`, `Micro`, `Macro` for the tower height.

output:
- The resulting table is written to excel, each company in a different worksheet


In [15]:

def process_company_data(input_csv, company_name, licensee_name, province, sheet_name, workbook_name):
    """Process one company CSV file and add a formatted worksheet to the given Excel workbook."""
    
    # Define column mappings and settings
    use_cols = ["Tier-4 id", "record_id", "area_name*", "latitude", "longitude",
                "structure_height", "tx_ant_azimuth", "licence_category*",
                "technology", "tx_power", "bandwidth"]
    group_unique_values = ['Tier-4 id', 'record_id', 'latitude', 'longitude', 'structure_height']
    group_repeated_values = ['licence_category*', 'tx_ant_azimuth', 'technology', 'bandwidth', 'tx_power']
    name_groupby = 'record_id'

    # Read input CSV
    df = pd.read_csv(input_csv, usecols=use_cols)

    # Group and aggregate
    results = []
    for group_name, group_df in df.groupby(name_groupby):
        row_result = {name_groupby: group_name}
        for col in group_unique_values:
            row_result[col] = group_df.iloc[0][col]
        for col in group_repeated_values:
            row_result[col] = '~'.join(group_df[col].astype(str).tolist())

        # Initialize accumulators
        row_result['4G_BW'] = 0
        row_result['5G_BW'] = 0
        for _, row in group_df.iterrows():
            tech = str(row['technology'])
            bw = float(row['bandwidth'])
            if tech == 'LTE':
                row_result['4G_BW'] += bw
            elif tech == '5G':
                row_result['5G_BW'] += bw 

        results.append(row_result)

    results = pd.DataFrame(results) #to be printed in the excel

    # Rename columns to destination names
    results = results.rename(columns={
        'Tier-4 id': 'Code',
        'record_id': 'Tower ID',
        'latitude': 'LATITUDE',
        'longitude': 'LONGITUDE',
        'structure_height': 'Height (m)',
        'tx_ant_azimuth': 'TX_ANT_AZI',
        'licence_category*': 'SERVICES',
        'technology': 'TECHNOLOGY',
        'tx_power': 'TX_PWR',
        'bandwidth': 'TR_BW_BLOCS'
    })

    # Add and transform fields
    results["Name"] = ' '
    results["Province"] = province
    results["Licensee"] = licensee_name
    results["Type"] = results["Height (m)"].apply(lambda x: 'Micro' if x < 10 else 'Macro')
    results["Micro"] = results["Type"].apply(lambda x: 1 if x == 'Micro' else '-')
    results["Macro"] = results["Type"].apply(lambda x: 1 if x == 'Macro' else '-')
    results["SERVICES"] = results["SERVICES"].str.replace("~", "|")
    results["TECHNOLOGY"] = results["TECHNOLOGY"].str.replace("LTE", "4G")

    # Reorder columns
    final_columns = ["Code", "Name", "Province", "Tower ID", "Licensee", "LATITUDE", "LONGITUDE",
                     "Height (m)", "Type", "Micro", "Macro", "TX_ANT_AZI", "SERVICES",
                     "TECHNOLOGY", "TR_BW_BLOCS", "4G_BW", "5G_BW", "TX_PWR"]
    results = results[final_columns].copy()

    # Append or create workbook
    if os.path.exists(workbook_name):
        with pd.ExcelWriter(workbook_name, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
            results.to_excel(writer, index=False, sheet_name=sheet_name)
    else:
        with pd.ExcelWriter(workbook_name, engine="openpyxl") as writer:
            results.to_excel(writer, index=False, sheet_name=sheet_name)

    # Apply styling
    wb = load_workbook(workbook_name)
    ws = wb[sheet_name]
    header_fill = PatternFill(start_color="002060", end_color="002060", fill_type="solid")  # dark blue
    header_font = Font(color="FFFFFF", bold=True)
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
    wb.save(workbook_name)

    print(f"✅ Added worksheet '{sheet_name}' for {company_name} to {workbook_name}")



## Process Multiple Companies into One Workbook

You can process multiple CSV files (e.g., TELUS, Videotron) and store them as separate sheets in the same workbook.


In [ ]:
output_workbook = "Telecom_Data_format2.xlsx"

process_company_data(
    input_csv="Rogers sample raw data.csv",
    company_name="Rogers",
    licensee_name="Rogers Communications Canada Inc.",
    province="QC",
    sheet_name="Rogers",
    workbook_name=output_workbook
)

process_company_data(
    input_csv="TELUS sample raw data.csv",
    company_name="TELUS",
    licensee_name="TELUS",
    province="QC",
    sheet_name="TELUS",
    workbook_name=output_workbook
)


process_company_data(
    input_csv="Videotron sample raw data.csv",
    company_name="Videotron",
    licensee_name="Videotron.",
    province="QC",
    sheet_name="Videotron",
    workbook_name=output_workbook
)



✅ Added worksheet 'Rogers' for Rogers to Telecom_Data_format2.xlsx
✅ Added worksheet 'TELUS' for TELUS to Telecom_Data_format2.xlsx
✅ Added worksheet 'Videotron' for Videotron to Telecom_Data_format2.xlsx


## Example: Format One Company

Process the Rogers CSV file and preview the formatted DataFrame.


### Notes on Transformation:



Example of output:
| Code  | Tower ID | LATITUDE | LONGITUDE | Height (m) | Type  | Micro | Macro | TX_ANT_AZI                | SERVICES                     | TECHNOLOGY                  | TR_BW_BLOCS              | 4G_BW | 5G_BW | TX_PWR              | ... |
|-------|----------|----------|-----------|------------|-------|-------|-------|----------------------------|------------------------------|-----------------------------|---------------------------|-------|-------|----------------------|-----|
| 4-048 | E0013    | 45.62212 | -73.0588  | 61         | Macro | -     | 1     | 20~170~290~20~170~290     | AWS|AWS|AWS|AWS|AWS|AWS      | 4G~4G~4G~4G~4G~4G           | 10~10~10~10~10~10        | 60.0  | 0     | 44.8~44.8~44.8~...   | ... |
| 4-030 | E0029    | 46.82311 | -71.4943  | 95         | Macro | -     | 1     | 0~120~240~0~120~240~...   | AWS|AWS|AWS|MBS|MBS|MBS|...   | 4G~4G~4G~4G~4G~4G~...        | 10~10~10~5~5~5~...       | 90.0  | 0     | 49.0~49.0~49.0~...   | ... |
| 4-030 | E0030    | 46.81697 | -71.1324  | 61         | Macro | -     | 1     | 30~140~260~30~140~260...  | AWS|AWS|AWS|MBS|MBS|MBS|...   | 4G~4G~4G~4G~4G~4G~...        | 10~10~10~5~5~5~...       | 90.0  | 0     | 46.0~46.0~46.0~...   | ... |
| 4-030 | E0031    | 46.56194 | -71.4420  | 78         | Macro | -     | 1     | 40~150~270~40~150~270...  | AWS|AWS|AWS|MBS|MBS|MBS|...   | 4G~4G~4G~4G~4G~4G~...        | 10~10~10~5~5~5~...       | ...   | ...   | ...                  | ... |


## Summary

- Raw CSV data is **grouped, aggregated, and reformatted** for each company.
- Output Excel workbook contains **one styled worksheet per company**.
- The approach is general and reusable for other telecom or similar datasets.
